In [9]:
import os
os.getcwd()


'd:\\'

In [8]:
# Load dataset with explicit null value mapping to handle missing data consistently
# 'Not Available' and 'unknown' strings are treated as NaN for proper aggregation later
import pandas as pd
retail_df = pd.read_csv('orders.csv',na_values=['Not Available', 'unknown'])
retail_df['Ship Mode'].unique()

FileNotFoundError: [Errno 2] No such file or directory: 'orders.csv'

In [ ]:
#Rename columns names and make them lower case and replace space with underscore
#retail_df.rename(columns={'Order Id':'order_id', 'City': 'city'}) 
#retail_df.columns=retail_df.columns.str.lower()
#retail_df.columns=retail_df.columns.str.replace(' ','_')
retail_df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [ ]:
# Calculate profit margin as the difference between sale price and cost
# This derived column enables profitability analysis across segments and time periods
retail_df['profit']=retail_df['sale_price']-retail_df['cost_price']
retail_df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [ ]:
# Convert order_date to datetime type for time-series operations and date-based filtering
# String dates cannot be sorted chronologically or used for time-based aggregations
retail_df['order_date']=pd.to_datetime(retail_df['order_date'],format="%Y-%m-%d")

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [ ]:
# Basic statistical summary to understand data distribution
retail_df.describe()

In [ ]:
#drop cost price, list price and discount percent columns
#retail_df.drop(['list_price', 'cost_price', 'discount_percent'], axis=1, inplace=True)
retail_df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [ ]:
#Connect to sql server 
import sqlalchemy as sal
from sqlalchemy import create_engine
engine = sal.create_engine('mssql://Suhith39\\SQLEXPRESS/mydatabase?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()

In [ ]:
# Persist cleaned retail data to SQL Server using append mode to preserve existing records
# This enables centralized data storage for business intelligence and reporting tools
retail_df.to_sql('df_orders', con=conn, index=False, if_exists = 'append')

38